# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 7 2022 9:17AM,238967,12,HH6151,Hush Hush,Released
1,Jun 7 2022 9:17AM,238966,10,6553642,"Citieffe, Inc.",Released
2,Jun 7 2022 9:13AM,238964,10,0085741547,ISDIN Corporation,Released
3,Jun 7 2022 9:13AM,238964,10,0085741588,ISDIN Corporation,Released
4,Jun 7 2022 9:13AM,238964,10,0085741589,ISDIN Corporation,Released
5,Jun 7 2022 9:13AM,238964,10,0085741614,ISDIN Corporation,Released
6,Jun 7 2022 9:13AM,238964,10,0085742575,ISDIN Corporation,Released
7,Jun 7 2022 9:13AM,238964,10,0085744162,ISDIN Corporation,Released
8,Jun 7 2022 9:13AM,238964,10,0085744181,ISDIN Corporation,Released
9,Jun 7 2022 9:13AM,238964,10,0085744180,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,238958,Released,7
43,238962,Released,1
44,238964,Released,64
45,238966,Released,1
46,238967,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238958,NaN,NaN,7.0
238962,NaN,NaN,1.0
238964,NaN,NaN,64.0
238966,NaN,NaN,1.0
238967,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238844,0.0,0.0,3.0
238857,0.0,1.0,0.0
238868,0.0,0.0,1.0
238873,0.0,0.0,1.0
238879,0.0,5.0,39.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238844,0,0,3
238857,0,1,0
238868,0,0,1
238873,0,0,1
238879,0,5,39


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238844,0,0,3
1,238857,0,1,0
2,238868,0,0,1
3,238873,0,0,1
4,238879,0,5,39


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238844,,,3
1,238857,,1,
2,238868,,,1
3,238873,,,1
4,238879,,5,39


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 7 2022 9:17AM,238967,12,Hush Hush
1,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc."
2,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation
66,Jun 7 2022 9:09AM,238955,10,ISDIN Corporation
101,Jun 7 2022 9:08AM,238962,18,Innova Softgel LLC
102,Jun 7 2022 9:02AM,238958,20,"Exact-Rx, Inc."
109,Jun 7 2022 8:59AM,238957,10,Beach Products Inc
110,Jun 7 2022 8:59AM,238956,20,Alumier Labs Inc.
132,Jun 7 2022 8:50AM,238954,10,"Herbivore Botanicals, LLC."
133,Jun 7 2022 8:48AM,238953,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 7 2022 9:17AM,238967,12,Hush Hush,,,1
1,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc.",,,1
2,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation,,,64
3,Jun 7 2022 9:09AM,238955,10,ISDIN Corporation,,,35
4,Jun 7 2022 9:08AM,238962,18,Innova Softgel LLC,,,1
5,Jun 7 2022 9:02AM,238958,20,"Exact-Rx, Inc.",,,7
6,Jun 7 2022 8:59AM,238957,10,Beach Products Inc,,,1
7,Jun 7 2022 8:59AM,238956,20,Alumier Labs Inc.,,,22
8,Jun 7 2022 8:50AM,238954,10,"Herbivore Botanicals, LLC.",,,1
9,Jun 7 2022 8:48AM,238953,10,Emerginnova,,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 9:17AM,238967,12,Hush Hush,1,,
1,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc.",1,,
2,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation,64,,
3,Jun 7 2022 9:09AM,238955,10,ISDIN Corporation,35,,
4,Jun 7 2022 9:08AM,238962,18,Innova Softgel LLC,1,,
5,Jun 7 2022 9:02AM,238958,20,"Exact-Rx, Inc.",7,,
6,Jun 7 2022 8:59AM,238957,10,Beach Products Inc,1,,
7,Jun 7 2022 8:59AM,238956,20,Alumier Labs Inc.,22,,
8,Jun 7 2022 8:50AM,238954,10,"Herbivore Botanicals, LLC.",1,,
9,Jun 7 2022 8:48AM,238953,10,Emerginnova,4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 9:17AM,238967,12,Hush Hush,1,,
1,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc.",1,,
2,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation,64,,
3,Jun 7 2022 9:09AM,238955,10,ISDIN Corporation,35,,
4,Jun 7 2022 9:08AM,238962,18,Innova Softgel LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 9:17AM,238967,12,Hush Hush,1.0,NaN,NaN
1,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation,64.0,NaN,NaN
3,Jun 7 2022 9:09AM,238955,10,ISDIN Corporation,35.0,NaN,NaN
4,Jun 7 2022 9:08AM,238962,18,Innova Softgel LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 9:17AM,238967,12,Hush Hush,1.0,0.0,0.0
1,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation,64.0,0.0,0.0
3,Jun 7 2022 9:09AM,238955,10,ISDIN Corporation,35.0,0.0,0.0
4,Jun 7 2022 9:08AM,238962,18,Innova Softgel LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3345069,173.0,4.0,11.0
12,477853,1.0,1.0,0.0
16,477814,6.0,20.0,0.0
17,477784,2.0,0.0,0.0
18,716733,3.0,0.0,0.0
19,1672379,10.0,15.0,0.0
20,1911299,112.0,35.0,0.0
21,716783,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3345069,173.0,4.0,11.0
1,12,477853,1.0,1.0,0.0
2,16,477814,6.0,20.0,0.0
3,17,477784,2.0,0.0,0.0
4,18,716733,3.0,0.0,0.0
5,19,1672379,10.0,15.0,0.0
6,20,1911299,112.0,35.0,0.0
7,21,716783,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,173.0,4.0,11.0
1,12,1.0,1.0,0.0
2,16,6.0,20.0,0.0
3,17,2.0,0.0,0.0
4,18,3.0,0.0,0.0
5,19,10.0,15.0,0.0
6,20,112.0,35.0,0.0
7,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,173.0
1,12,Released,1.0
2,16,Released,6.0
3,17,Released,2.0
4,18,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,18,19,20,21
Status,,,,,,,,
Completed,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,4.0,1.0,20.0,0.0,0.0,15.0,35.0,1.0
Released,173.0,1.0,6.0,2.0,3.0,10.0,112.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,18,19,20,21
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,1.0,20.0,0.0,0.0,15.0,35.0,1.0
2,Released,173.0,1.0,6.0,2.0,3.0,10.0,112.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,18,19,20,21
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,1.0,20.0,0.0,0.0,15.0,35.0,1.0
2,Released,173.0,1.0,6.0,2.0,3.0,10.0,112.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()